In [2]:
import transformers
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
import torch
 
import numpy as np
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from collections import defaultdict
from textwrap import wrap
 
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
 
%matplotlib inline
%config InlineBackend.figure_format='retina'
 
sns.set(style='whitegrid', palette='muted', font_scale=1.2)
 
HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]
sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))
 
rcParams['figure.figsize'] = 12, 8
 
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
df = pd.read_csv("D:/Codefield/CODE_PYTHON/NLP/Prob1/ABSA/Restaurants_Train_v2.csv")

In [4]:
df.shape

(3693, 6)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3693 entries, 0 to 3692
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          3693 non-null   int64 
 1   content     3693 non-null   object
 2   AspectTerm  3693 non-null   object
 3   polarity    3693 non-null   object
 4   from        3693 non-null   int64 
 5   to          3693 non-null   int64 
dtypes: int64(3), object(3)
memory usage: 173.2+ KB


In [6]:
def to_sentiment(polarity):
    polarity = str(polarity)
    if polarity == 'negative':
        return 0
    elif polarity == 'neutral':
        return 1
    else:
        return 2

    
df['sentiment'] = df.polarity.apply(to_sentiment)
 
class_names = ['negative', 'neutral', 'positive']


In [7]:
PRE_TRAINED_MODEL_NAME= 'bert-base-cased'

In [8]:
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

In [9]:
MAX_LEN = 160

In [10]:
class GPReviewDataset(Dataset):
 
    def __init__(self, reviews, targets, tokenizer, max_len):
        self.reviews = reviews
        #self.review_aspects = review_aspects
        self.targets = targets
        self.tokenizer = tokenizer
        self.max_len = max_len
 
    def __len__(self):
        return len(self.reviews)
 
    def __getitem__(self, item):
        review = str(self.reviews[item])
        target = self.targets[item]
 
        encoding = self.tokenizer.encode_plus(
            review,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            pad_to_max_length=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
 
        return {
            'review_text': review,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'targets': torch.tensor(target, dtype=torch.long)
        }

In [11]:
df_train, df_test = train_test_split(
    df,
    test_size=0.1,
    random_state=RANDOM_SEED
)
df_val, df_test = train_test_split(
    df_test,
    test_size=0.5,
    random_state=RANDOM_SEED
)
 
df_train.shape, df_val.shape, df_test.shape

((3323, 7), (185, 7), (185, 7))

In [12]:
def create_data_loader(df, tokenizer, max_len, batch_size):
    ds = GPReviewDataset(
        reviews=df.content.to_numpy(),
        #review_aspects = df.AspectTerm.to_numpy(),
        targets=df.sentiment.to_numpy(),
        tokenizer=tokenizer,
        max_len=max_len
    )
 
    return DataLoader(
        ds,
        batch_size=batch_size,
        num_workers=0
    )
 
BATCH_SIZE = 4
 
train_data_loader = create_data_loader(df_train, tokenizer, MAX_LEN, BATCH_SIZE)
val_data_loader = create_data_loader(df_val, tokenizer, MAX_LEN, BATCH_SIZE)
test_data_loader = create_data_loader(df_test, tokenizer, MAX_LEN, BATCH_SIZE)

In [13]:
data = next(iter(train_data_loader))
data.keys()

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
D:\Python\Python311\Lib\site-packages\transformers\tokenization_utils_base.py:2377: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


dict_keys(['review_text', 'input_ids', 'attention_mask', 'targets'])

In [14]:
print(data['input_ids'].shape)
print(data['attention_mask'].shape)
print(data['targets'].shape)

torch.Size([4, 160])
torch.Size([4, 160])
torch.Size([4])


In [15]:
data['input_ids']

tensor([[  101,  1109,  1178,  4931,  2546,  1420,  1108,  1103,  2564,  1120,
          1103,  2927,   119,   102,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,  

In [16]:
bert_model = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [17]:
class SentimentClassifier(nn.Module):
 
    def __init__(self, n_classes):
        super(SentimentClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME, output_hidden_states=True, output_attentions=True, return_dict=False)
        self.drop = nn.Dropout(p=0.3)
        self.out = nn.Linear(self.bert.config.hidden_size, n_classes)
 
    def forward(self, input_ids, attention_mask):
        _, pooled_output,_,_ = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        output = self.drop(pooled_output)
        return self.out(output)

In [18]:
model = SentimentClassifier(len(class_names))
model = model.to(device)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [19]:
#input_ids = data['input_ids'].to(device)
#attention_mask = data['attention_mask'].to(device)
 
#print(input_ids.shape) # batch size x seq length
#print(attention_mask.shape) # batch size x seq length

In [20]:
#torch.nn.functional.softmax(model(input_ids, attention_mask), dim=1)

In [21]:
EPOCHS = 10
 
optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)
total_steps = len(train_data_loader) * EPOCHS
 
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)
 
loss_fn = nn.CrossEntropyLoss().to(device)

D:\Python\Python311\Lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [22]:
def train_epoch(
    model,
    data_loader,
    loss_fn,
    optimizer,
    device,
    scheduler,
    n_examples
  ):
    model = model.train()
 
    losses = []
    correct_predictions = 0
 
    for d in data_loader:
        input_ids = d["input_ids"].to(device)
        #input_ids = input_ids.squeeze(0)
        attention_mask = d["attention_mask"].to(device)
        #attention_mask = attention_mask.squeeze(0)
        #print(input_ids.shape) # batch size x seq length
        #print(attention_mask.shape) # batch size x seq length
        targets = d["targets"].to(device)
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
        )
 
        _, preds = torch.max(outputs, dim=1)
        loss = loss_fn(outputs, targets)
 
        correct_predictions += torch.sum(preds == targets)
        losses.append(loss.item())
 
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
 
    return correct_predictions.double() / n_examples, np.mean(losses)

In [23]:
def eval_model(model, data_loader, loss_fn, device, n_examples):
    model = model.eval()
 
    losses = []
    correct_predictions = 0
 
    with torch.no_grad():
        for d in data_loader:
            input_ids = d["input_ids"].to(device)
            attention_mask = d["attention_mask"].to(device)
            targets = d["targets"].to(device)
 
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask
            )
            _, preds = torch.max(outputs, dim=1)
 
            loss = loss_fn(outputs, targets)
 
            correct_predictions += torch.sum(preds == targets)
            losses.append(loss.item())
 
    return correct_predictions.double() / n_examples, np.mean(losses)

In [24]:
%%time
 
history = defaultdict(list)
best_accuracy = 0
 
for epoch in range(EPOCHS):
 
    print(f'Epoch {epoch + 1}/{EPOCHS}')
    print('-' * 10)
 
    train_acc, train_loss = train_epoch(
        model,
        train_data_loader,
        loss_fn,
        optimizer,
        device,
        scheduler,
        len(df_train)
    )
 
    print(f'Train loss {train_loss} accuracy {train_acc}')
 
    val_acc, val_loss = eval_model(
        model,
        val_data_loader,
        loss_fn,
        device,
        len(df_val)
    )
 
    print(f'Val   loss {val_loss} accuracy {val_acc}')
    print()
 
    history['train_acc'].append(train_acc)
    history['train_loss'].append(train_loss)
    history['val_acc'].append(val_acc)
    history['val_loss'].append(val_loss)
 
    if val_acc > best_accuracy:
        torch.save(model.state_dict(), 'best_model_state_2.bin')
        best_accuracy = val_acc

Epoch 1/10
----------
Train loss 0.7603523036763161 accuracy 0.7005717724947337
Val   loss 0.7255871009993109 accuracy 0.7081081081081081

Epoch 2/10
----------
Train loss 0.5823111337013924 accuracy 0.8161300030093289
Val   loss 1.0831179628693597 accuracy 0.7189189189189189

Epoch 3/10
----------
Train loss 0.4987990102826233 accuracy 0.8564550105326513
Val   loss 1.3714010282295777 accuracy 0.6864864864864865

Epoch 4/10
----------
Train loss 0.4609108183933052 accuracy 0.8712007222389407
Val   loss 1.5176627825130709 accuracy 0.7027027027027027

Epoch 5/10
----------
Train loss 0.42803569395182944 accuracy 0.8775203129702076
Val   loss 1.6410929571844617 accuracy 0.7027027027027027

Epoch 6/10
----------
Train loss 0.40811830978738606 accuracy 0.8868492326211255
Val   loss 1.6173955467480057 accuracy 0.7135135135135136

Epoch 7/10
----------
Train loss 0.3938438171894451 accuracy 0.8883538970809509
Val   loss 1.6096501434327133 accuracy 0.7027027027027027

Epoch 8/10
----------
Tra

In [25]:
##没有什么用的分割线
input_ids = data['input_ids'].to(device)
attention_mask = data['attention_mask'].to(device)
 
print(input_ids.shape) # batch size x seq length
print(attention_mask.shape) # batch size x seq length

torch.Size([4, 160])
torch.Size([4, 160])


In [26]:
# plt.plot(history['train_acc.item()'],[1], label='train accuracy')
# plt.plot(history['val_acc.item()'],[1], label='validation accuracy')
# plt.title('Training history')
# plt.ylabel('Accuracy')
# plt.xlabel('Epoch')
# plt.legend()
# plt.ylim([0, 1]);
# plt.xlim([0,10]);
# plt.show()

In [27]:
history['train_acc']

[tensor(0.7006, dtype=torch.float64),
 tensor(0.8161, dtype=torch.float64),
 tensor(0.8565, dtype=torch.float64),
 tensor(0.8712, dtype=torch.float64),
 tensor(0.8775, dtype=torch.float64),
 tensor(0.8868, dtype=torch.float64),
 tensor(0.8884, dtype=torch.float64),
 tensor(0.8896, dtype=torch.float64),
 tensor(0.8923, dtype=torch.float64),
 tensor(0.8947, dtype=torch.float64)]

In [28]:
history['val_acc']

[tensor(0.7081, dtype=torch.float64),
 tensor(0.7189, dtype=torch.float64),
 tensor(0.6865, dtype=torch.float64),
 tensor(0.7027, dtype=torch.float64),
 tensor(0.7027, dtype=torch.float64),
 tensor(0.7135, dtype=torch.float64),
 tensor(0.7027, dtype=torch.float64),
 tensor(0.7081, dtype=torch.float64),
 tensor(0.7081, dtype=torch.float64),
 tensor(0.7135, dtype=torch.float64)]

In [30]:
#调用训练好的模型
model = SentimentClassifier(len(class_names))
model.load_state_dict(torch.load('best_model_state_2.bin'))
model = model.to(device)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [31]:
test_acc, _ = eval_model(
    model,
    test_data_loader,
    loss_fn,
    device,
    len(df_test)
)
test_acc.item()

0.7567567567567568

In [32]:
def get_predictions(model, data_loader):
    model = model.eval()
 
    review_texts = []
    predictions = []
    prediction_probs = []
    real_values = []
 
    with torch.no_grad():
        for d in data_loader:
 
            texts = d["review_text"]
            input_ids = d["input_ids"].to(device)
            attention_mask = d["attention_mask"].to(device)
            targets = d["targets"].to(device)
 
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask
            )
            _, preds = torch.max(outputs, dim=1)
 
            review_texts.extend(texts)
            predictions.extend(preds)
            prediction_probs.extend(outputs)
            real_values.extend(targets)
 
    predictions = torch.stack(predictions).cpu()
    prediction_probs = torch.stack(prediction_probs).cpu()
    real_values = torch.stack(real_values).cpu()
    return review_texts, predictions, prediction_probs, real_values

In [33]:
y_review_texts, y_pred, y_pred_probs, y_test = get_predictions(
    model,
    test_data_loader
)

In [34]:
idx = 2
 
review_text = y_review_texts[idx]
true_sentiment = y_test[idx]
pred_df = pd.DataFrame({
    'class_names': class_names,
    'values': y_pred_probs[idx]
})
 
print("\n".join(wrap(review_text)))
print()
print(f'True sentiment: {class_names[true_sentiment]}')

No one asked what was wrong as we left with nothing touched on our
plates.

True sentiment: neutral


In [35]:
review_text = "Hate you!!!"

In [36]:
encoded_review = tokenizer.encode_plus(
    review_text,
    max_length=MAX_LEN,
    add_special_tokens=True,
    return_token_type_ids=False,
    pad_to_max_length=True,
    return_attention_mask=True,
    return_tensors='pt',
)

In [37]:
input_ids = encoded_review['input_ids'].to(device)
attention_mask = encoded_review['attention_mask'].to(device)
 
output = model(input_ids, attention_mask)
_, prediction = torch.max(output, dim=1)
 
print(f'Review text: {review_text}')
print(f'Sentiment  : {class_names[prediction]}')

Review text: Hate you!!!
Sentiment  : positive
